In [1]:
from datahelpers.constants import iden, ye, ai, ps, up, dn, ar, ni, cexp, qcexp, nw, wi, dist, rdist, pm, \
                                    cpop, cden, ct, affs, aus
from os.path import expanduser, join
import pandas as pd
from bm_support.add_features import generate_feature_groups
from bm_support.supervised_aux import split_three_way
from bm_support.supervised import select_features_dict
from bm_support.add_features import normalize_columns
from bm_support.supervised_aux import study_sample, metric_selector
from functools import partial
from multiprocessing import Pool
from copy import deepcopy
import warnings
from numpy.random import RandomState
import gzip
import pickle
import argparse
from sklearn.model_selection import train_test_split
from bm_support.supervised import simple_stratify
from bm_support.supervised import select_features_dict, logit_pvalue, report_metrics
from sklearn.ensemble import RandomForestRegressor

warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 1
%aimport bm_support.add_features
%aimport bm_support.supervised_aux
%aimport bm_support.supervised

In [2]:
an_version = 12

# origin = 'litgw'
# version = 1

# origin = 'lit'
# version = 8

origin = 'gw'
version = 11

# model_type = 'rf'
# seed0 = 17
n_trials = 1
n_subtrials = 2
n_estimators = 17
# n_estimators = 55
datapath = None
# seed0 = 13
seed0 = 1
n_jobs = 1
verbose = True
model_type = 'lr'
model_type = 'rf'


min_log_alpha = -1
max_log_alpha = 1
log_reg_dict = {'min_log_alpha': min_log_alpha, 'max_log_alpha': max_log_alpha}

In [3]:
eps = 0.2
upper_exp, lower_exp = 1 - eps, eps
# thrs = [-1e-8, lower_exp, upper_exp, 1.0001e0]
if datapath:
    col_families = generate_feature_groups(expanduser(join(datapath, 'v12_columns.txt')))
else:
    col_families = generate_feature_groups(expanduser('~/data/kl/columns/v12_columns.txt'))

if verbose:
    print('Number of col families: {0}. Keys: {1}'.format(len(col_families), sorted(col_families.keys())))

col_families = {k: v for k, v in col_families.items() if 'future' not in k}
if verbose:
    print('Number of col families (excl. future): {0}. Keys: {1}'.format(len(col_families),
                                                                         sorted(col_families.keys())))

columns_interest = [x for sublist in col_families.values() for x in sublist]
if datapath:
    df_path = expanduser(join(datapath, '{0}_{1}_{2}.h5'.format(origin, version, an_version)))
else:
    df_path = expanduser('~/data/kl/final/{0}_{1}_{2}.h5'.format(origin, version, an_version))

df = pd.read_hdf(df_path, key='df')


# mask: literome - mask out a specific interaction
mask_lit = (df[up] == 7157) & (df[dn] == 1026)

# mask:  interaction with more than 3 claims
thr = 3
mask_len_ = (df.groupby(ni).apply(lambda x: x.shape[0]) > thr)
mask_max_len = df[ni].isin(mask_len_[mask_len_].index)
print(mask_max_len.shape[0], sum(mask_max_len))

# mask : interactions which are between
eps_window_mean = 0.1
mean_col = 0.5
mask_exp = ((df[cexp] <= lower_exp - eps_window_mean) | (df[cexp] >= upper_exp + eps_window_mean)
            # | ((df2[cexp] <= (mean_col + eps_window_mean)) & (df2[cexp] >= (mean_col - eps_window_mean)))
            )

feature_dict = deepcopy(col_families)
# families = ['affiliations_affind', 'affiliations_comm_size', 'affiliations_ncomms',
#             'affiliations_ncomponents', 'affiliations_size_ulist', 'affiliations_suppind',
#             'ai', 'ar', 'authors_affind', 'authors_comm_size', 'authors_ncomms', 'authors_ncomponents',
#             'authors_size_ulist', 'authors_suppind', 'cden', 'citations',
#             'cite_count', 'cpop', 'delta_year', 'ksst', 'lincscomm_size', 'lincssame_comm',
#             'litgwcsize_dn', 'litgwcsize_up',
#             'litgweff_comm_size', 'litgwsame_comm', 'nhi', 'past_affind',
#             'past_comm_size', 'past_ncomms', 'past_ncomponents',
#             'past_size_ulist', 'past_suppind', 'pre_affs', 'pre_authors', 'time']
families = ['affiliations_comm_size',
            'ai', 'ar', 'cden', 'citations',
            'cite_count', 'cpop', 'delta_year', 'ksst', 'lincscomm_size', 'lincssame_comm',
            'litgweff_comm_size', 'litgwsame_comm', 'nhi', 'past_affind',
            'past_comm_size', 'time']

feature_dict = {k: v for k, v in feature_dict.items() if k in families}

trial_features = [x for sublist in feature_dict.values() for x in sublist]

feature_dict_inv = {}
for k, v in feature_dict.items():
    feature_dict_inv.update({x: k for x in v})

# mask: not nulls in trial features
masks = []
for c in trial_features:
    masks.append(df[c].notnull())

mask_notnull = masks[0]
for m in masks[1:]:
    mask_notnull &= m

print('Experimental mask len {0}'.format(sum(mask_exp)))
print('Number of trial features: {0}'.format(len(trial_features)))
print('Number of notnull entries (over all features): {0} from {1}'.format(sum(mask_notnull), mask_notnull.shape))

if origin != 'gw':
    mask_agg = mask_notnull & ~mask_lit
else:
    mask_agg = mask_notnull

dfw = df.loc[mask_agg].copy()

#metric to optimize for
mm = 'accuracy'
# mm = 'precision'

target_column = 'litgw_comm_im_undir_wei_pNone_eff_comm_size'

nmax = 5000

rns = RandomState(seed0)
seeds = rns.randint(nmax, size=n_trials)

df_train, df_test, df_valid = split_three_way(dfw, seed0, target_column)

['cpop', 'cden', 'ksst', 'nhi', 'affiliations_affind', 'authors_affind', 'future_affind', 'past_affind', 'affiliations_suppind', 'authors_suppind', 'future_suppind', 'past_suppind', 'affiliations_comm_size', 'authors_comm_size', 'future_comm_size', 'past_comm_size', 'affiliations_ncomms', 'authors_ncomms', 'future_ncomms', 'past_ncomms', 'affiliations_ncomponents', 'authors_ncomponents', 'future_ncomponents', 'past_ncomponents', 'affiliations_size_ulist', 'authors_size_ulist', 'future_size_ulist', 'past_size_ulist', 'pre_authors', 'pre_affs']
{'cpop': 8, 'cden': 8, 'ksst': 8, 'nhi': 2, 'affiliations_affind': 4, 'authors_affind': 4, 'future_affind': 4, 'past_affind': 4, 'affiliations_suppind': 4, 'authors_suppind': 4, 'future_suppind': 4, 'past_suppind': 4, 'affiliations_comm_size': 4, 'authors_comm_size': 4, 'future_comm_size': 4, 'past_comm_size': 4, 'affiliations_ncomms': 4, 'authors_ncomms': 4, 'future_ncomms': 4, 'past_ncomms': 4, 'affiliations_ncomponents': 4, 'authors_ncomponents

In [4]:
model_type = 'lrg'
model_type = 'rfr'

if model_type == 'rf' or model_type == 'rfr':
    param_dict = {'n_estimators': n_estimators, 'max_features': None, 'n_jobs': 1}
else:
    param_dict = {'n_jobs': 1}

if model_type == 'rf' or model_type == 'rfr':
    param_dict['random_state'] = 17
elif model_type == 'lr':
    param_dict['C'] = 1.0

In [5]:
col_pool = [target_column]

In [6]:
feature_dict_new = {k: list(v) for k, v in feature_dict.items() if not any([c in v for c in col_pool])}
len(feature_dict), len(feature_dict_new)

(17, 16)

In [ ]:
#metric to optimize for
mm = 'accuracy'
# mm = 'precision'

target_column = 'litgw_comm_im_undir_wei_pNone_eff_comm_size'

col_pool = [target_column]

feature_dict_new = {k: list(v) for k, v in feature_dict.items() if not any([c in v for c in col_pool])}
len(feature_dict), len(feature_dict_new)

study_sample_flag = True
# study_sample_flag = False
model_type = 'lrg'
model_type = 'rfr'
verbose=True

if study_sample_flag:
    r = study_sample(dfw=dfw, target=target_column, feature_dict=feature_dict_new, metric_mode=mm,
                       model_type=model_type,
                       n_subtrials=n_subtrials, n_estimators=n_estimators,
                       log_reg_dict=log_reg_dict, verbose=verbose, seed=seed0)
else:
    nmax = 5000
    
    rns = RandomState(seed0)
    seeds = rns.randint(nmax, size=n_trials)
    
    df_train, df_test, df_valid = split_three_way(dfw, seed0, target_column)
    
    if model_type == 'rf' or model_type == 'rfr':
        param_dict = {'n_estimators': n_estimators, 'max_features': None, 'n_jobs': 1}
    else:
        param_dict = {'n_jobs': 1}
    
    param_dict['random_state'] = 17
    
    if model_type == 'rf' or model_type == 'rfr':
        param_dict = {'n_estimators': n_estimators, 'max_features': None, 'n_jobs': 1}
    else:
        param_dict = {'n_jobs': 1}
    
    if model_type == 'rf' or model_type == 'rfr':
        param_dict['random_state'] = 17
    elif model_type == 'lr':
        param_dict['C'] = 1.0
    
    
    
    r = select_features_dict(df_train, df_test, target_column, feature_dict_new, model_type='rfr', max_features_consider=8,
                             metric_mode='accuracy', mode_scores=None,
                             metric_uniform_exponent=0.5, eps_improvement=1e-6,
                             model_dict=param_dict, verbose=True)
    
    print(r)

model_dict {'n_estimators': 17, 'max_features': None, 'n_jobs': 1, 'random_state': 235}
nf: 1 cfeature: lincs_comm_im_undir_unwei_p... metric: -0.011 metric_improv: 100.00 %
Fractional potential improvement: 0.06266838038560452
nf: 2 cfeature: lincs_comm_ml_undir_wei_pNo... metric: -0.011 metric_improv: 6.27 %
Fractional potential improvement: 0.015259073768838949
nf: 3 cfeature: litgw_comm_ml_undir_unwei_p... metric: -0.010 metric_improv: 1.53 %
Fractional potential improvement: 0.03373099514820699
nf: 4 cfeature: litgw_comm_im_dir_wei_pNone... metric: -0.010 metric_improv: 3.37 %
Fractional potential improvement: -0.0004962529924058545
Terminating early: no improvement.
model_dict {'n_estimators': 17, 'max_features': None, 'n_jobs': 1, 'random_state': 5192}
nf: 1 cfeature: lincs_comm_im_undir_unwei_p... metric: -0.011 metric_improv: 100.00 %


In [76]:
r

(['lincs_comm_im_undir_unwei_p95_eff_comm_size',
  'lincs_comm_ml_undir_wei_pNone_eff_comm_size',
  'litgw_comm_ml_undir_unwei_p95_same_comm',
  'litgw_comm_im_dir_wei_pNone_same_comm',
  'len_flag',
  'ksst1'],
 [-0.011155945110876343,
  -0.010562614208895147,
  -0.010342602382690096,
  -0.010060589922301551,
  -0.010025177965262921,
  -0.010016958874797003],
 [{'corr': 0.7903745331195964,
   'mse': 0.011155945110876343,
   'main_metric': -0.011155945110876343},
  {'corr': 0.8374195704891597,
   'mse': 0.010562614208895147,
   'main_metric': -0.010562614208895147},
  {'corr': 0.853030037726166,
   'mse': 0.010342602382690096,
   'main_metric': -0.010342602382690096},
  {'corr': 0.8628943816236623,
   'mse': 0.010060589922301551,
   'main_metric': -0.010060589922301551},
  {'corr': 0.8610588531763788,
   'mse': 0.010025177965262921,
   'main_metric': -0.010025177965262921},
  {'corr': 0.8609635826302688,
   'mse': 0.010016958874797003,
   'main_metric': -0.010016958874797003}],
 Random